Class 16: Agenda
   
   1. Lasso/Ridge Review
   2. RF: 
       a. Dendrograms
       b. Feature Importance
   3. Classification
       a. Logistic Regression (SVM Support Vector Machine was written in the 90s in C)
           i. Sigmoid
           ii. Api
       b. Random Forest Cluster
   4. Grid Search?
  

In [46]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

In [24]:
df = pd.read_csv('/Users/swllms/DAT-10-14-SW/class material/Unit3/Data/housing.csv')

In [25]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import cross_val_score

In [4]:
X = df.iloc[:, :-1]
y = df['PRICE']

In [39]:
X.mean()

CRIM       8.326673e-17
ZN         3.466704e-16
INDUS     -3.016965e-15
CHAS       3.999875e-16
NOX        3.167427e-15
RM        -1.258809e-14
AGE       -1.158274e-15
DIS        7.308603e-16
RAD       -1.068535e-15
TAX        6.534079e-16
PTRATIO   -1.084420e-14
B          8.117354e-15
LSTAT     -6.494585e-16
dtype: float64

In [8]:
#MUST Standardize before you run! 
X = (X - X.mean())/X.std()

In [5]:
lasso = Lasso()

In [9]:
lasso.fit(X, y)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [10]:
lasso.coef_

array([-0.        ,  0.        , -0.        ,  0.        , -0.        ,
        2.71542789, -0.        , -0.        , -0.        , -0.        ,
       -1.34428304,  0.18036988, -3.54677609])

If coef is set 0 (Lasso) it is functionally removed from your data set better for very large messy data. Ridge REDUCES to near 0 and more conservative. 

Coef mean the same as basic linear reg. think of this as a form of regression that chooses what features to focus on for the predictions. 

In [15]:
alpha = 10

In [16]:
l1_term = alpha * np.sum(np.abs(lasso.coef_))

In [17]:
l1_term #this is the penalty term! 

77.86856908757778

In [20]:
alphas = np.logspace (-4, 4, 9) #Sensible range of testing Alpha, this block tests alpha
lasso_scores = []

for alpha in alphas:
    lasso.set_params(alpha=alpha)
    scores = cross_val_score(estimator=lasso, X=X, y=y, cv=10)
    lasso_scores.append((np.mean(scores), alpha))

In [21]:
lasso_scores
#0.1 would have been the best version for alpha

[(0.20262012278999347, 0.0001),
 (0.20343535169466956, 0.001),
 (0.21144430759385185, 0.01),
 (0.2407890782467927, 0.1),
 (0.1807548507575551, 1.0),
 (-1.2860830508551744, 10.0),
 (-1.2860830508551744, 100.0),
 (-1.2860830508551744, 1000.0),
 (-1.2860830508551744, 10000.0)]

## Random Forrests

Tend to be a bit more accurate then linear models but DS were saying that it was not clear the information provided by ensembles. Feature importance and Dendrograms are used to figure out what causes what. 

In [28]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [32]:
rf.fit(X, y)

/Users/swllms/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [33]:
rf.feature_importances_

array([0.04415707, 0.00060074, 0.00499874, 0.00050157, 0.01755777,
       0.37504468, 0.01591818, 0.07291872, 0.00376635, 0.01501034,
       0.01391251, 0.01132919, 0.42428412])

In [35]:
feats = pd.DataFrame({
    'Featrues': X.columns,
    'Importance': rf.feature_importances_
}).sort_values(by='Importance', ascending = False)

These numbers will sum up to a 1 and the closest thing your will get to a coef but not the same thing, all numbers are positive.

Indirectly tells you the realative importance of each value by looking at score (r squared value) and randomly rearrange the colmn value anc compare the new score to your model score

In [36]:
feats 

,Featrues,Importance
12,LSTAT,0.424284
5,RM,0.375045
7,DIS,0.072919
0,CRIM,0.044157
4,NOX,0.017558
6,AGE,0.015918
9,TAX,0.015010
10,PTRATIO,0.013913
11,B,0.011329
2,INDUS,0.004999


In [37]:
rf.score(X, y)

0.9769181552281025

You can create trees based on thresholds of importance. you can see more details doing the following. 

In [38]:
feats['Importance'].cumsum() #cumilative sum to show you what is critical from %

12    0.424284
5     0.799329
7     0.872248
0     0.916405
4     0.933962
6     0.949881
9     0.964891
10    0.978803
11    0.990133
2     0.995131
8     0.998898
1     0.999498
3     1.000000
Name: Importance, dtype: float64

Dendrograms is jonathans favorite topic and hard to find details online. 
Pairs perfectly with Random Forest! 

Nothing that is done automatically so takes a few extra steps. 

Pearson's Correlation = Linear Association
Spearmans Correlation = Rank Correlation 

See seperate notebook in prompts section. Will be posted later tonight 12/11/19

## Classifications

Predicting discreet categories instead of continious numbers 
ie: boy/girl, green/blue/black

Logistic Regression works like linear model Using Sigmoid function takes continuous numbers and tranforms them into prob between 0 and 1 1/1+e**-x e**-x = y=MX+b

The results are the odds that this events that this will happen. 

In [41]:
def sigmoid(x):
    return 1 / (1 +np.exp(-x))

In [42]:
sigmoid (0) 
# 50 % Chance this would happen

0.5

In [43]:
sigmoid () 

0.9990889488055994

In [48]:
import pandas as pd

In [51]:
train = pd.read_csv('/Users/swllms/Desktop/retonightsexitticket/train.csv')
test = pd.read_csv('/Users/swllms/Desktop/retonightsexitticket/test.csv')

In [52]:
X_train = train.loc[:, ['Sex', 'Pclass']]
y_train = train['Survived']
X_test = test.loc[:, ['Sex', 'Pclass']]

In [53]:
X_train = pd.get_dummies(X_train, columns=['Sex', 'Pclass'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['Sex', 'Pclass'], drop_first=True)

Drop first removes the first column! Always a good idea when you one hot incode with minimal impact to your data or with catagorical data

Reduces the number of colms and gives your a better picture of the impact. 

The first column that your drop gets wrapped up in the inercept term (the prediction that you make if all other information is 0)

In [56]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='liblinear')

In [57]:
logreg.get_params() 
#C is alpha, alpha is C however it is inverted (0.1 = 100)
#higher vlaues singal lowest strength or regularization and vice versa
#penalty default is set to L2 (Ridge and Lasso are bulit in you can specify l1 or l2)

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'warn',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [58]:
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [59]:
logreg.predict(X_test)
#0 = passagner died
#1 = passanger lived

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [60]:
logreg.predict_proba(X_test) #Must also do this step
#first number predicts the chance that number was 0, second predicts if they lived

array([[0.89503473, 0.10496527],
       [0.41462909, 0.58537091],
       [0.75093089, 0.24906911],
       [0.89503473, 0.10496527],
       [0.41462909, 0.58537091],
       [0.89503473, 0.10496527],
       [0.41462909, 0.58537091],
       [0.75093089, 0.24906911],
       [0.41462909, 0.58537091],
       [0.89503473, 0.10496527],
       [0.89503473, 0.10496527],
       [0.60521334, 0.39478666],
       [0.11295975, 0.88704025],
       [0.75093089, 0.24906911],
       [0.11295975, 0.88704025],
       [0.20028547, 0.79971453],
       [0.75093089, 0.24906911],
       [0.89503473, 0.10496527],
       [0.41462909, 0.58537091],
       [0.41462909, 0.58537091],
       [0.60521334, 0.39478666],
       [0.89503473, 0.10496527],
       [0.11295975, 0.88704025],
       [0.60521334, 0.39478666],
       [0.11295975, 0.88704025],
       [0.89503473, 0.10496527],
       [0.11295975, 0.88704025],
       [0.89503473, 0.10496527],
       [0.60521334, 0.39478666],
       [0.89503473, 0.10496527],
       [0.

In [61]:
logreg.predict_proba(X_test)[:, 1]

array([0.10496527, 0.58537091, 0.24906911, 0.10496527, 0.58537091,
       0.10496527, 0.58537091, 0.24906911, 0.58537091, 0.10496527,
       0.10496527, 0.39478666, 0.88704025, 0.24906911, 0.88704025,
       0.79971453, 0.24906911, 0.10496527, 0.58537091, 0.58537091,
       0.39478666, 0.10496527, 0.88704025, 0.39478666, 0.88704025,
       0.10496527, 0.88704025, 0.10496527, 0.39478666, 0.10496527,
       0.24906911, 0.24906911, 0.58537091, 0.58537091, 0.39478666,
       0.10496527, 0.58537091, 0.58537091, 0.10496527, 0.10496527,
       0.10496527, 0.39478666, 0.10496527, 0.79971453, 0.88704025,
       0.10496527, 0.39478666, 0.10496527, 0.88704025, 0.58537091,
       0.39478666, 0.24906911, 0.79971453, 0.88704025, 0.24906911,
       0.10496527, 0.10496527, 0.10496527, 0.10496527, 0.88704025,
       0.10496527, 0.24906911, 0.10496527, 0.58537091, 0.39478666,
       0.79971453, 0.58537091, 0.39478666, 0.39478666, 0.88704025,
       0.58537091, 0.10496527, 0.58537091, 0.39478666, 0.88704

In [62]:
logreg.intercept_

array([2.06085883])

In [63]:
logreg.coef_

array([[-2.48809433, -0.67634771, -1.7159975 ]])

In [66]:
coeffs = pd.DataFrame({
    'Variable': X_train.columns, 
    'Weight': logreg.coef_[0]
})
coeffs

,Variable,Weight
0,Sex_male,-2.488094
1,Pclass_2,-0.676348
2,Pclass_3,-1.715997


In [68]:
output = X_train.dot(logreg.coef_.T) + logreg.intercept_
output

,0
0,-2.143233
1,2.060859
2,0.344861
3,2.060859
4,-2.143233
5,-2.143233
6,-0.427235
7,-2.143233
8,0.344861
9,1.384511


In [69]:
sigmoid (output)
#the probablity! #Ground zero of nural networks
#natural range for sigmoid is - 7 to 7
#REMEMBER: higher vlaues singal lowest strength or regularization and vice versa

,0
0,0.104965
1,0.887040
2,0.585371
3,0.887040
4,0.104965
5,0.104965
6,0.394787
7,0.104965
8,0.585371
9,0.799715


## Random Forest Classifer

Works in a manner that almost idential to their Regressor Counterpart
Classify a sample by takin a majority vote of the each leave balue 
Basic syntax is the same

Takes the majority vote of that leaf
ie if 10 people are on a leaf and 6 lived and one died it would spit out that you would have a prob of 60% 

All other information for random trees remains the same. 

In [74]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [75]:
rf.fit(X_train, y_train)

/Users/swllms/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [76]:
rf.feature_importances_

array([0.72745472, 0.0267825 , 0.24576278])

# See slides for the remainder of the notes! 